In [1]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

# !ls "/content/drive/My Drive"
!cp '/content/drive/My Drive/AI/datasets.zip' datasets.zip
# # !cp '/content/drive/My Drive/AI/model_output_xception_overfitting-20201014T084949Z-001.zip' trained_model.zip
!unzip -qq datasets.zip
# !unzip -qq trained_model.zip 


Mounted at /content/drive


In [ ]:
# !cp '/content/drive/My Drive/AI/model_output_xception_overfitting-20201014T084949Z-001.zip' trained_model.zip
# !unzip -qq trained_model.zip 

In [2]:
import numpy as np
import pandas as pd
import cv2
from sklearn.utils._joblib import Parallel
from sklearn.utils._joblib import delayed

import random
from skimage import img_as_ubyte
import os
from skimage.util import random_noise

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization , Activation, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator # new!
from matplotlib import pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.initializers import Zeros, glorot_normal
from tensorflow.keras.callbacks import TensorBoard ,ReduceLROnPlateau, ModelCheckpoint
import os
from skimage import io 
import skimage as sk
from skimage.transform import rotate, AffineTransform, warp


In [ ]:
def random_rotation(image):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return rotate(image, random_degree)

def random_noise(image):
    # add random noise to the image
    return random_noise(image)

def blurring(iamge):
      return cv2.GaussianBlur(iamge, (9,9),0)

def contrast(image):
    return 2*image

def add_gaussian_noise(image):
  # Adding Gaussian noise to image
  common_type = np.float32 # Make noise and image of the same type
  gnoise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.4, dtype=common_type)
  image_type_converted = tf.image.convert_image_dtype(image, dtype=common_type, saturate=False)
  noisy_image = tf.add(image_type_converted, gnoise)
  return noisy_image

# def inverse_color(image):
#     return sk.util.invert(image)

def affine_transform(image):
    _tf = AffineTransform(shear=-0.5)
    return warp(image, _tf, order=1, preserve_range=True, mode='wrap')

  
available_transformations = np.array([random_rotation, random_noise, blurring, contrast , affine_transform])

In [ ]:
def iamge_generator(x, y):
    indices= np.random.choice(5,2, replace = False)
    funcs = available_transformations[indices]
    f_idx = 0
    rows = []
    for f in funcs:
        image_to_transform = sk.io.imread(data_train_dir + x)
        transformed_image = f(image_to_transform)
        filename = "aug_"  + str(f_idx)  + "_" + x
        rows.append([filename , y])
#         files.append(filename)
        sk.io.imsave(data_train_dir + filename, transformed_image)
        f_idx = f_idx + 1
        
    return rows

In [ ]:
train_label_aug = pd.read_csv(data_train_label_aug)
# file_col = train_label.ID.map(lambda x: str(x)+".png").to_numpy()
file_col = train_label.ID.to_numpy()
label_col = train_label.Label.map(lambda x: str(x)).to_numpy()
train_label = train_label.assign(ID=file_col, Label=label_col)
print(train_label.shape)

(3492, 2)


In [4]:
train_label = pd.read_csv(data_train_label)
file_col = train_label.ID.map(lambda x: str(x)+".png").to_numpy()
# file_col = train_label.ID.to_numpy()
label_col = train_label.Label.map(lambda x: str(x)).to_numpy()
train_label = train_label.assign(ID=file_col, Label=label_col)
print(train_label.shape)

(1164, 2)


In [ ]:
print(train_label.head())

      ID Label
0  0.png     1
1  1.png     0
2  2.png     1
3  3.png     1
4  4.png     1


In [ ]:
all_res = Parallel(n_jobs=None)(
        delayed(iamge_generator)
        (x,y) for x,y in zip(file_col, label_col))

In [ ]:
print(train_label.shape)
all_res = np.array(all_res)
print(all_res.shape)
all_res = all_res.reshape(1164*2,2)
print(all_res.shape)


(1164, 2)
(1164, 2, 2)
[['aug_0_0.png' '1']
 ['aug_1_0.png' '1']
 ['aug_0_1.png' '0']
 ...
 ['aug_1_1162.png' '1']
 ['aug_0_1163.png' '1']
 ['aug_1_1163.png' '1']]


In [ ]:
print(train_label.shape)
for i in all_res:    
    new_row = {'ID':i[0], 'Label':i[1]}
    #append row to the dataframe
    train_label = train_label.append(new_row, ignore_index=True)
print(train_label.shape)


In [5]:

BATCH_SIZE=32
IMG_SIZE=(512, 512)
w_init = glorot_normal()
b_init = Zeros()

In [3]:
#parameter

data_train_label = "/content/train_label.csv"
data_train_label_aug = "/content/augment_train_label.csv"
data_train_dir = "/content/train_image/"
data_test_dir = "/content/test_image/"
output_dir = 'model_output/'
output_training= output_dir + 'check_point'
output_logs = output_dir + 'logs'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(output_training):
    os.makedirs(output_training)
if not os.path.exists(output_logs):
    os.makedirs(output_logs)

In [7]:
modelcheckpoint = ModelCheckpoint(filepath=output_training +"/resnet50_{epoch:03d}-{val_loss:.3f}-{val_accuracy:.3f}.h5", 
                                  monitor='val_accuracy',
                                  mode = 'max',
                                  save_best_only=True)
tensorboard = TensorBoard(output_logs)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              mode='min',verbose = 1,
                              patience=6, min_lr=0.00001)

In [18]:

datagen1=ImageDataGenerator(rescale=1./255)

data_train1 = datagen1.flow_from_dataframe(dataframe=train_label, directory=data_train_dir,
                                                      x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=42,
                                                      target_size=IMG_SIZE, batch_size=64)



Found 1164 validated image filenames belonging to 3 classes.


In [ ]:
datagen_no_augmentation=ImageDataGenerator(rescale=1./255, validation_split=0.1)

data_train_no_augmentation = datagen_no_augmentation.flow_from_dataframe(dataframe=train_label, directory=data_train_dir, subset='training',
                                                      x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=42,
                                                      target_size=IMG_SIZE, batch_size=BATCH_SIZE)

data_validation_no_augmentation = datagen_no_augmentation.flow_from_dataframe(dataframe=train_label, directory=data_train_dir, subset='validation',
                                                        x_col="ID", y_col="Label", shuffle = True, class_mode="categorical",seed=42,
                                                        target_size=IMG_SIZE, batch_size=BATCH_SIZE)




Found 3143 validated image filenames belonging to 3 classes.
Found 349 validated image filenames belonging to 3 classes.


In [ ]:
datagen_augmentation_noise=ImageDataGenerator(rescale=1./255,
                                        brightness_range=[0.2,1],
                                        zoom_range=0.1,
                                        preprocessing_function = add_noise,
                                        validation_split=0.3)

data_train_noise = datagen_augmentation_noise.flow_from_dataframe(dataframe=train_label, directory=data_train_dir, subset='training',
                                                      x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=42,
                                                      target_size=IMG_SIZE, batch_size=BATCH_SIZE)

data_validation_noise = datagen_augmentation_noise.flow_from_dataframe(dataframe=train_label, directory=data_train_dir, subset='validation',
                                                        x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=42,
                                                        target_size=IMG_SIZE, batch_size=BATCH_SIZE)


Found 815 validated image filenames belonging to 3 classes.
Found 349 validated image filenames belonging to 3 classes.


In [ ]:
datagen_augmentation=ImageDataGenerator(rescale=1./255,
                                        # shear_range=0.2,
                                        rotation_range=10,
                                        brightness_range=[0.3,0.9],
                                        zoom_range=0.2,
                                        validation_split=0.2)

data_train_augmentation = datagen_augmentation.flow_from_dataframe(dataframe=train_label, directory=data_train_dir, subset='training',
                                                      x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=42,
                                                      target_size=IMG_SIZE, batch_size=BATCH_SIZE)

data_validation_augmentation = datagen_augmentation.flow_from_dataframe(dataframe=train_label, directory=data_train_dir, subset='validation',
                                                        x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=42,
                                                        target_size=IMG_SIZE, batch_size=BATCH_SIZE)



Found 932 validated image filenames belonging to 3 classes.
Found 232 validated image filenames belonging to 3 classes.


In [ ]:
testgen=ImageDataGenerator(rescale=1./255)

data_test=testgen.flow_from_directory(directory=data_test_dir, 
                                      shuffle = False, target_size=IMG_SIZE, class_mode='categorical', batch_size=1)

Found 292 images belonging to 1 classes.


In [15]:
resnet50 = ResNet50(include_top=False,
              weights="imagenet",
              input_shape=(512,512,3),
              pooling = "max")
for layer in resnet50.layers:
    layer.trainable = False

model = Sequential()
model.add(resnet50)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(160, kernel_initializer=w_init, bias_initializer=b_init, kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [16]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)


loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.05,
    reduction="auto",
    name="categorical_crossentropy",
)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [13]:
model.optimizer.lr = 0.0001

In [19]:
model.fit(data_train1, epochs=40, verbose=1)

Epoch 1/40
19/19 [==============================] - 45s 2s/step - loss: 1.3251 - accuracy: 0.5816
Epoch 2/40
19/19 [==============================] - 45s 2s/step - loss: 1.3249 - accuracy: 0.5722
Epoch 3/40
19/19 [==============================] - 45s 2s/step - loss: 1.3228 - accuracy: 0.5730
Epoch 4/40
19/19 [==============================] - 45s 2s/step - loss: 1.2968 - accuracy: 0.6031
Epoch 5/40
19/19 [==============================] - 47s 2s/step - loss: 1.2861 - accuracy: 0.5765
Epoch 6/40
19/19 [==============================] - 45s 2s/step - loss: 1.2628 - accuracy: 0.5979
Epoch 7/40
19/19 [==============================] - 45s 2s/step - loss: 1.2934 - accuracy: 0.5816
Epoch 8/40
19/19 [==============================] - 45s 2s/step - loss: 1.2812 - accuracy: 0.5816
Epoch 9/40
19/19 [==============================] - 45s 2s/step - loss: 1.2517 - accuracy: 0.6057
Epoch 10/40
19/19 [==============================] - 45s 2s/step - loss: 1.2665 - accuracy: 0.5936
Epoch 11/40
19/19 [

In [ ]:
# model = tf.keras.models.load_model(output_training +"/vgg19_927.h5")
model.evaluate(data_test, verbose=2)


In [ ]:
print(tf.keras.backend.eval(model.optimizer.lr))

1e-05


In [ ]:

model.evaluate(data_validation_no_augmentation, verbose=2)

11/11 - 8s - loss: 0.1062 - accuracy: 0.9628


[0.1062137633562088, 0.9627507328987122]

In [ ]:
predicted_classes = np.argmax(model.predict(data_test), axis = -1)

In [ ]:
test_id = data_test.filenames
new_test_id = [x.replace('.png', '').replace('test_image/', '') for x in test_id]


In [ ]:
evaluation = pd.DataFrame({'ID':new_test_id, 'Label':predicted_classes})
evaluation.to_csv(output_dir + "vgg19_augmentation_2_evaluation_submission.csv", index=False) # Public Score: 0.76555
evaluation.head()

,ID,Label
0,0,0
1,1,1
2,10,0
3,100,2
4,101,1


In [ ]:
model.save(output_dir + "xception_noise_model.h5")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_output/check_point/assets


In [ ]:
!cp -r 'model_output' '/content/drive/My Drive/AI/vgg19' 

In [ ]:
!rm -r 'model_output'

In [4]:
!cp -r '/content/drive/My Drive/AI/test_image' .

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model(output_dir + "xception_noise_model.h5")

# Show the model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 16, 16, 2048)      20861480  
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 2048)        0         
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dropout (Dropout)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 30)                3932190   
_________________________________________________________________
batch_normalization_4 (Batch (None, 30)                120       
_________________________________________________________________
activation (Activation)      (None, 30)                0

In [ ]:
np.argmax(model.predict(data_test), axis = -1)

array([0, 1, 0, 2, 1, 2, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 2, 0,
       1, 0, 2, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 0, 2, 2, 1, 2, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0,
       1, 1, 0, 0, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 0, 1, 1, 1, 2,
       2, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 0, 0, 1, 0, 0, 2, 1, 2,
       1, 2, 1, 1, 1, 2, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1,
       0, 1, 1, 2, 2, 0, 2, 0, 1, 2, 1, 0, 1, 2, 2, 1, 1, 2, 0, 0, 2, 0,
       0, 0, 2, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 2, 0, 0, 2,
       2, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 2, 1, 0, 2, 1, 2, 1, 2,
       1, 0, 1, 1, 2, 1, 1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 0, 1, 0, 0, 1, 1,
       2, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 1, 1,
       1, 0, 1, 0, 2, 1, 2, 1, 2, 0, 0, 1, 2, 1, 1, 1, 1, 0, 1, 0, 2, 1,
       0, 1, 2, 1, 1, 0])

In [ ]:
np.argmax(new_model.predict(data_test), axis = -1)

array([0, 1, 0, 2, 1, 2, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 2, 0,
       1, 0, 2, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 0, 2, 2, 1, 2, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0,
       1, 1, 0, 0, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 0, 1, 1, 1, 2,
       2, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 0, 0, 1, 0, 0, 2, 1, 2,
       1, 2, 1, 1, 1, 2, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1,
       0, 1, 1, 2, 2, 0, 2, 0, 1, 2, 1, 0, 1, 2, 2, 1, 1, 2, 0, 0, 2, 0,
       0, 0, 2, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 2, 0, 0, 2,
       2, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 2, 1, 0, 2, 1, 2, 1, 2,
       1, 0, 1, 1, 2, 1, 1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 0, 1, 0, 0, 1, 1,
       2, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 1, 1,
       1, 0, 1, 0, 2, 1, 2, 1, 2, 0, 0, 1, 2, 1, 1, 1, 1, 0, 1, 0, 2, 1,
       0, 1, 2, 1, 1, 0])

In [ ]:
print(tf.keras.backend.eval(model.optimizer.lr))

1e-04


In [ ]:
model.optimizer.lr = 0.01
print(tf.keras.backend.eval(new_model.optimizer.lr))

0.01
